### Stažení zdrojového souboru .xml do lokálního adresáře

In [1]:
from urllib.request import urlretrieve

url = "https://portal.isoss.cz/opendata/ISoSS_Opendata_EOSM.xml"
local_filename = "ISoSS_Opendata_EOSM.xml"
urlretrieve(url, local_filename)

('ISoSS_Opendata_EOSM.xml', <http.client.HTTPMessage at 0x260fb3a5ee0>)

### Import knihoven, parsování xml a vytvoření dataframe

In [2]:
import pandas as pd
import xml.etree.cElementTree as ET

tree = ET.parse("ISoSS_Opendata_EOSM.xml")
root = tree.getroot()

result = []
radek = []

for child in root:
        # nazev_obce
        radek.append(child[2][1].text)
        # nazev (místa)
        radek.append(child[3].text.strip().lower())
        # popis (místa)
        radek.append(child[4].text)
        # uvazek_vyse
        radek.append(child[5].text)
        # platova_trida
        radek.append(child[6].text)
        # prace_od
        radek.append(child[7].text)
        # min_vzdelani
        radek.append(child[8].text)
        # nabidka_platna_do
        radek.append(child[9].text)
        # 12 doba_urcita
        if child[12].tag.find("Doba") != -1:
            radek.append(child[12][1].text)
        else:
            radek.append("doba neurčitá")
        # 17 1 zadost_podani_organizace
        for elem in child.iter():
            if elem.tag.find("ZadostPodaniMisto") != -1:
                radek.append(elem[0].text)
        result.append(radek)
        radek = []

df = pd.DataFrame(data=result, columns=["nazev_obce", 
                                        "nazev", 
                                        "oddeleni_odbor", 
                                        "uvazek_vyse", 
                                        "platova_trida", 
                                        "prace_od", 
                                        "min_vzdelani", 
                                        "nabidka_platna_do",
                                        "doba_ne/urcita",
                                        "zadost_podani_organizace"
                                       ])
df.head()

,nazev_obce,nazev,oddeleni_odbor,uvazek_vyse,platova_trida,prace_od,min_vzdelani,nabidka_platna_do,doba_ne/urcita,zadost_podani_organizace
0,Praha,3038 - vrchní vládní rada,"Oddělení koncepcí, strategií a programů",1.0,14,2024-09-02,magisterský studijní program,2024-08-14,doba neurčitá,Úřad vlády ČR
1,Praha,2163 - vrchní referent,Oddělení vnitřní bezpečnosti,1.0,10,2024-09-15,vyšší odb. vzdělání/bakalářský st. prog.,2024-08-16,doba neurčitá,Úřad vlády České republiky
2,Praha,2466 - vládní rada,Oddělení interního auditu a kontroly,1.0,13,2024-09-01,magisterský studijní program,2024-08-16,doba neurčitá,Úřad vlády České republiky
3,Praha,2465 - vládní rada,Oddělení interního auditu a kontroly,1.0,13,2024-09-01,magisterský studijní program,2024-08-16,doba neurčitá,Úřad vlády České republiky
4,Praha,3093 - vládní rada,Odd. koord. a fin. protidr. politiky,1.0,13,2024-09-16,magisterský studijní program,2024-08-21,doba neurčitá,Úřad vlády České republiky


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   nazev_obce                465 non-null    object
 1   nazev                     465 non-null    object
 2   oddeleni_odbor            465 non-null    object
 3   uvazek_vyse               465 non-null    object
 4   platova_trida             465 non-null    object
 5   prace_od                  465 non-null    object
 6   min_vzdelani              465 non-null    object
 7   nabidka_platna_do         465 non-null    object
 8   doba_ne/urcita            465 non-null    object
 9   zadost_podani_organizace  465 non-null    object
dtypes: object(10)
memory usage: 36.5+ KB


### Vyhledávání pomocí pozice

In [ ]:
pozice = input("Pozice: ")
pozice = pozice.strip().lower()
df_final = df[df["nazev"].str.contains(pozice)].sort_values(by=["platova_trida"], ascending=False)
df_final